In [1]:
# necessary imports
import os, json
import pandas as pd
import numpy as np
import glob
from tqdm.notebook import tqdm
pd.set_option('display.max_columns', None)
import sys
sys.path.append('../scripts/')
from twitter import get_hashtags
import ast

In [2]:
# read lda dates for hashtags
dates_df = pd.read_csv('../../data/BTW17_Twitter/peaks/hashtags_dates.csv')
dates_df = dates_df[~(dates_df['lda_dates']=='[]')].reset_index()
dates_df.drop(columns=['Unnamed: 0', 'index'], axis=1, inplace=True)
dates_df.head()

,hashtag,lda_dates
0,afd,"['2017-09-16', '2017-09-17', '2017-09-18', '20..."
1,afghanistan,"['2017-08-22', '2017-08-23', '2017-08-24', '20..."
2,armut,"['2017-08-28', '2017-08-29', '2017-08-30', '20..."
3,bayern,"['2017-09-11', '2017-09-12', '2017-09-13', '20..."
4,berlin,"['2017-09-02', '2017-09-03', '2017-09-04', '20..."


In [20]:
# set to *.json to load all
path_to_json = '../../data/BTW17_Twitter/tweets/*.json'
file_list = glob.glob(path_to_json)
hashtag_list = dates_df['hashtag'].tolist()
lda_dates_list = dates_df['lda_dates'].tolist()

created_at_list = []
id_list = []
text_list = []
user_list = []
extended_tweet_list = []
retweeted_status_list = []
tags_list = []

# iterate through files create output csv
for index in tqdm(range(len(file_list))):
    file = file_list[index]
    with open(file, 'r') as f:
        
        # read df
        df = pd.read_json(f)
        # apply function get hashtags
        df['tags'] = df['entities'].apply(get_hashtags)
        
        # explode tags to rows and drop na values
        df = df.explode('tags')
        df = df[df['tags'].notna()]
        
        # change datetime to date and lower all hashtags
        df['created_at'] = pd.to_datetime(df['created_at']).dt.date
        df['created_at'] = df['created_at'].apply(str)
        df['tags'] = df['tags'].apply(lambda x: str(x).lower())
                
        for i in range(len(dates_df)):
            hashtag = hashtag_list[i]
            lda_dates = ast.literal_eval(lda_dates_list[i])
            temp_df = df[(df['tags']==hashtag)&(df['created_at'].isin(lda_dates))]
            
            if len(temp_df)==0:
                pass
            else:
                # append data to lists
                created_at_list.append(temp_df['created_at'].tolist())
                id_list.append(temp_df['id'].tolist())
                text_list.append(temp_df['text'].tolist())
                user_list.append(temp_df['user'].tolist())
                extended_tweet_list.append(temp_df['extended_tweet'].tolist())
                retweeted_status_list.append(temp_df['retweeted_status'].tolist())
                tags_list.append(temp_df['tags'].tolist())

In [27]:
# create df
output_df = pd.DataFrame(data={'created_at': created_at_list, 'id': id_list, 'text': text_list,
                               'user': user_list, 'extended_tweet': extended_tweet_list,
                               'retweeted_status': retweeted_status_list, 'tags': tags_list})

# explode df after setting fake column
output_df['A'] = 1
output_df = output_df.set_index(['A']).apply(pd.Series.explode).reset_index()
output_df.drop(columns='A', inplace=True)

output_df.head()

,created_at,id,text,user,extended_tweet,retweeted_status,tags
0,2017-08-02,892871750018621440,"Endlich sagts mal einer, immerhin im Leitartik...","{'id': 52024480, 'id_str': '52024480', 'name':...",NaN,NaN,dobrindt
1,2017-08-02,892872839916859392,"RT @DJanecek: Endlich sagts mal einer, immerhi...","{'id': 3338824444, 'id_str': '3338824444', 'na...",NaN,{'created_at': 'Wed Aug 02 22:16:28 +0000 2017...,dobrindt
2,2017-08-02,892874663499632640,"RT @DJanecek: Endlich sagts mal einer, immerhi...","{'id': 4828089006, 'id_str': '4828089006', 'na...",NaN,{'created_at': 'Wed Aug 02 22:16:28 +0000 2017...,dobrindt
3,2017-08-03,892911669633966080,#Diesel #Dobrindt's schützende Hand über Autok...,"{'id': 22900494, 'id_str': '22900494', 'name':...",NaN,NaN,dobrindt
4,2017-08-25,901122414809100288,RT @DuHugonotte: Nun kommt auch der #Staatssen...,"{'id': 4830265798, 'id_str': '4830265798', 'na...",NaN,{'created_at': 'Fri Aug 25 05:32:42 +0000 2017...,ard


In [28]:
# save to csv
path_file = '../../data/BTW17_Twitter/lda/lda_tweets.csv'
output_df.to_csv(path_file)